# **Pipeline d'analyse des données de séquençage - Projet BILL**

*Arnaud Charlery-Adèle et Daphné Navratil - M1 IMHE et Bioinformatique*

### **Contexte**

Dans le cadre du projet BILL, nous nous sommes interessés à l'effet d’un choc thermique froid sur l'évolution in vitro du Cyprinid Herpesvirus 3. Des cellules de cerveau de carpe commune ont été cultivées in vitro et inculées avec la souche italienne KHV-I du virus. Le virus a été passé en série 50 fois. Au passage 25, les cultures ont subi un choc thermique à 15°C. Pour répondre à notre problématique, le génome du virus issu des passages successifs avant et après choc thermique (P10, P20, P30, P40, P50) a été séquencé. Pour analyser les données, nous avons utilisé une pipeline décrite dans ce jupyter notebook.

La pipeline a été réalisée avec SnakeMake 6.14.0 avec python 3.7 par Arnaud Soulier. Elle permet de traiter les données issues du séquençage long reads Oxford Nanopore et est définie par un fichier Snakefile qui permet de lancer une suite de commandes BASH.

La pipeline a été lancé individuellement pour chaque barcode et prend en entrer les fichiers fastq concaténés par barcode issus du séquençage.








### **Les différentes commandes lancées par la pipeline et les logiciels auxquelles elles appartiennent sont les suivants :**






### **Seqkit 2.1.0** [1]

seqkit seq -m {trim} {input.raw} -o {output.trimed} 2> {log.out}

La première étape de cette pipeline est d’utiliser Seqkit seq pour retirer des fichiers fastq les reads avec une taille inférieure à 1000 pb. La commande prend en entrer un fichier fastq.gz et donne en sortie un fichier fastq.gz.
L'option -m permet de filtrer les reads inférieurs à une taille précisée, ici 1000, tandis que l'option -o permet de spécifier le nom du fichier de sortie.


### **Minimap2 2.17** [2]

minimap2 --MD -ax map-ont -t {threads} {input.ref} {input.read} -o {output.sam} 2> {log.out}

Minimap2 alligne les reads issus du séquençage sur le génome de référence de la souche japonaise du virus, KHV-J. Le fichier pris en entrée est le fichier de sortie de Seqkit seq et est donc au format fastq.gz. Le fichier de sortie est un fichier sam.

### **Samtools 1.9**

**Samtools flagstat** [3]

samtools flagstat -@ {threads} {input.sorted} > {output.stats} 2> {log.out}

Samtools flagstat utilise le fichier sam obtenu en sortie de l'alignement réalisé par minimap2. La commande permet d'obtenir de statistiques sur l'alignement en indiquant, en autre, le nombre de reads correspondant à chaque flag. Par exemple : le nombre de reads n'ayant pas mappés sur la référence, nombre de reads qui ont mappés partiellement etc. Le fichier de sortie est au format flagstat.

**samtools view** [4]

samtools view -uhbS -@ {threads} -F 4 {input.sam} -o {output.mapped} 2> {log.out}

Samtools convertit le fichier sam issu de minimap2 en fichier bam et de retirer les reads ne s'alignant pas sur le génome de référence.

**samtools sort [5]**

samtools sort -l 0 -@ {threads} {input.mapped} -o {output.sorted} 2> {log.out}

A partir des fichiers bam, samtools sort sert à trier les reads selon leur position le long du chromosome pour permettre le fonctionnement de samtools index. Le fichier de sortie est au format bam.

**samtools index [6]**

samtools index -@ {threads} {input.sorted} 2> {log.out}

Sniffles utilise le fichier bam indexé et prépare les reads pour l’appel de variant réalisé avec Sniffles en indexant les régions d'intérêt. Le fichier de sortie est un fichier bam.bai.

### **sniffles 1.0.11 [7]**

sniffles -t {threads} -i {input.sorted} --minsvlen 10 --minsupport 10 -v {output.vcf} 2> {log.out}

Sniffles réalise l'appel de variant à partir du fichier bam de sortie de samtools sort et du fichier bam indexé. Il permet d'obtenir un fichier vcf qui liste les substitutions et indels, pour des long reads. Seules les mutations dont la longueur est supérieure ou égale à 10pb (précisé par l'option --minsvlen), et présentes sur au moins 10 reads (précisé par l'option --minsupport) ont été sélectionnées. L'option -i précise le fichier pris en entrée qui est ici le fichier bam issu de smatools sort.


### **Deeptools 3.5.0 [8]**

**bamCoverage**

bamCoverage -p {threads} -b {input.sorted} -of "bedgraph" --effectiveGenomeSize {genomeSize} --normalizeUsing RPGC -o {output.bedgraph} 2> {log.out}

Permet d'obtenir à partir du fichier bam obtenu à la sortie de samtools sort la couverture de séquençage dans un fichier appelé Bedgraph.

**plotCoverage**

plotCoverage -p {threads} -b {input.sorted} --smartLabels --plotFileFormat pdf -o {output.pdf} 2> {log.out}

En prenant en entrée le fichier bam donné par la commande samtools sort, plotCoverage donne la profondeur de séquençage, c'est à dire le nombre de reads qui soutiennent un nuclétotide dans l'alignement. Le fichier de sortie est un pdf (précisé par l'option --plotFileFormat) qui contient des graphiques plotCov. 


### **Fichiers de sortie de la pipeline**

A l'issue de la pipeline, on obtient donc:


*   Un fichier vcf donné par Sniffles
*   Un fichier bedgraph donné par bamCovergae
*   Un fichier plotCov.pdf donné par plotCoverage
*   Un fichier flagstat donné par Samtools flagstat



## **Traitement des fichiers vcf:**


Pour comparer quelles mutations sont communes ou non entre les différents passages, nous avons utilisé une commande Bcftools ainsi qu'un programme créé par Elliot Butz avec python 3.

### **Bcftools 1.17 [9]**

bcftools isec -n=*nombre_de_fichiers* -c all *fichiers_vcf_à_comparer*

La commande bcftools isec permet de comparer les différents fichiers vcf donnés en paramètre. Avec l'option -n, on précise le nombre de fichiers que l'on souhaite comparer. Tanids que l'option -c all permet d'obtenir les variants communs entre les fichiers. Nous avons donc comparer l'ensemble des fichiers vcf obtenus pour les passage 10, 20, 30, 40 et 50.

### **Programme python 3 par Elliot Butz**

**La fonction compare_vcfs()**

La classe set() permet de calculer les différences entre deux ensembles avec une complexité O(len(s)) d'après la documentation (https://python-reference.readthedocs.io/en/latest/docs/sets/op_difference.html).
La fonction prend en entrée deux fichiers vcf à comparer et donne:

*   l'ensemble des mutations présentent dans le premier fichier vcf et non dans le deuxième.
*   l'ensemble des mutations présentent dans le second fichier vcf et non dans le premier.

La fonction parcourt les fichiers et liste leurs mutations dans deux ensembles distincs. Avec des fonctionnalités de la classe set(), les mutations qui ne sont pas communes aux deux ensembles sont listés dans deux nouveaux ensembles qui sont ceux doner à la sortie de la fonction.

Néanmoins cette fonctione ne fonctionne par pour l'ensemble des fichiers vcf.


In [ ]:
def compare_vcfs(vcf_file1, vcf_file2):
    
    # Initialisation des ensembles qui contiendront les mutations des fichiers d'entrée. 
    mutations1 = set() #mutation1 contiendra les fichiers présent dans le premier fichier vcf.
    mutations2 = set() #mutation2 contiendra les fichiers présent dans le second fichier vcf.

    
    # Extraction des mutations du premier fichier d'entrée.
    with open(vcf_file1, 'r') as f1: 
        
        # Parcourt du fichier ligne par ligne.
        for line in f1: 
            
            # Les lignes prises en compte sont celles qui ne commencent pas par #.
            if line[0] != '#':
                
                # Ajout de l'identifiant de la mutation à l'ensemble mutation1.
                mutations1.add(tuple(line.strip().split()[:2]))
    
    # Extraction des mutations du second fichier d'entrée. La méthode utilisée est exactement la même
    # que pour le premier fichier.
    with open(vcf_file2, 'r') as f2:
        for line in f2:
            if line[0] != '#':
                mutations2.add(tuple(line.strip().split()[:2]))

    not_in_file2 = mutations1 - mutations2 # Retire à l'ensemble des mutations qui se trouvent dans le premier fichier
                                           # l'ensemble des mutations qui se trouvent dans le second.

    not_in_file1 = mutations2 - mutations1 # Retire à l'ensemble des mutations qui se trouvent dans le second fichier
                                           # l'ensemble des mutations qui se trouvent dans le premier.

    return (not_in_file1, not_in_file2) # Renvoie les ensembles des mutations qui différencient les deux fichiers.


  #Nous vousinvitons à tester notre fonction. Pour cela, il suffit d'indiquer les 
#chemins d'accès à deux fichiers vcf à l'aide des deux lignes suivantes.

vcf1 = '/content/P20.C1_RB03.trimed1000.mapped.sorted.vcf'
vcf2 = '/content/P30.C1_RB04.trimed1000.mapped.sorted.vcf'

print("Les mutations présentes dans vcf1 mais pas dans vcf2 sont : ",compare_vcfs(vcf1, vcf2)[0], ".")
print("Les mutations présentes dans vcf2 mais pas dans vcf1 sont : ",compare_vcfs(vcf1, vcf2)[1], ".")
  Les mutations présentes dans vcf1 mais pas dans vcf2 sont :  {('AP008984.1', '187879'), ('AP008984.1', '216398'), ('AP008984.1', '203393'), ('AP008984.1', '91777'), ('AP008984.1', '76121'), ('AP008984.1', '216445'), ('AP008984.1', '14372'), ('AP008984.1', '60348'), ('AP008984.1', '133372'), ('AP008984.1', '16050'), ('AP008984.1', '143580'), ('AP008984.1', '75785'), ('AP008984.1', '11325'), ('AP008984.1', '216397'), ('AP008984.1', '11334'), ('AP008984.1', '75786'), ('AP008984.1', '272125'), ('AP008984.1', '205799'), ('AP008984.1', '284172'), ('AP008984.1', '240065'), ('AP008984.1', '240068'), ('AP008984.1', '131128'), ('AP008984.1', '91094')} .
Les mutations présentes dans vcf2 mais pas dans vcf1 sont :  {('AP008984.1', '22553'), ('AP008984.1', '216447'), ('AP008984.1', '49275'), ('AP008984.1', '272124'), ('AP008984.1', '29449'), ('AP008984.1', '95871'), ('AP008984.1', '165009'), ('AP008984.1', '91195'), ('AP008984.1', '91201'), ('AP008984.1', '155432'), ('AP008984.1', '31249'), ('AP008984.1', '186412'), ('AP008984.1', '90631'), ('AP008984.1', '53428'), ('AP008984.1', '22309'), ('AP008984.1', '47119'), ('AP008984.1', '270735'), ('AP008984.1', '273975')} .

**La fonction count_diffs()**

Elle compte le nombre de mutations non communes aux deux fichiers vcf pris en entré. Elle donne en sortie : [texte du lien](https://)

*   la taille de l'ensemble des mutations qui apparaissent dans le premier vcf pris en entrée mais pas dans le second.
*   la taille de l'ensemble des mutations qui apparaissent dans le second vcf pris en entrée mais pas dans le premier.

compare_vcf créé deux ensembles contenant les mutations qui différencient les deux vcf d'entrée et renvoie la tailles de ces deux ensembles.

In [ ]:
def count_diffs(vcf_file1, vcf_file2):
    
    # Initialisation des ensembles qui contiendront les mutations des fichiers d'entrée. 
    OnlyIn1 = compare_vcfs(vcf_file1, vcf_file2)[0] #OnlyIn1 contient les fichiers présent dans le premier fichier vcf.
    OnlyIn2 = compare_vcfs(vcf_file1, vcf_file2)[1] #OnlyIn2 contient les fichiers présent dans le second fichier vcf.

    NumberNotInFile2 = len(OnlyIn1) # Calcule la taille de l'ensemble des mutations qui se trouvent dans  
                                    # mutations1 mais pas dans mutations2.

    NumberNotInFile1 = len(OnlyIn2) # Calcule la taille de l'ensemble des mutations qui se trouvent dans  
                                    # mutations2 mais pas dans mutations1.

    return (NumberNotInFile2, NumberNotInFile1) # Renvoie des nombres de mutations qui différencient les deux fichiers.  # Voici une cellule pour tester la fonction count_diffs. Elle fait appel à l'initialisation des variables vcf1 et vcf2 deux cellules au dessus.

print("Il y a ", count_diffs(vcf1, vcf2)[0], " mutations présentes dans vcf1 qui n'apparaissent pas dans vcf2.")
print("Il y a ", count_diffs(vcf1, vcf2)[1], " mutations présentes dans vcf2 qui n'apparaissent pas dans vcf1.")  Il y a  23  mutations présentes dans vcf1 qui n'apparaissent pas dans vcf2.
Il y a  18  mutations présentes dans vcf2 qui n'apparaissent pas dans vcf1.

**La fonction count_indels2()**

Nous nous sommes demandé si les chocs thermiques à p25 a eut un impact sur la proportion d'insertions et de délétions détectées.

La fonction prend en entré un fichier vcf et donne en sortie le nombre d'insertions et de délétions présentes dans le fichier.

La fonction parcourt les lignes du fichier du fichier vcf et ignore les lignes qui comment par "#". La fonction parcours les différents colonnes du fichier vcf à la recherche de la chaîne "INS" dans la troisième colonne. Lorsque la chaîne est trouvée le compteur d'insertion augmente de 1 et dans le cas contraire c'est le compteur de déletions qui augmente de 1.

In [ ]:
def count_indels2(vcf_file):
    
    # Initialisation des compteurs d'inserstions de délétions.
    insertions = 0
    deletions = 0

    # Ouverture du fichier.
    with open(vcf_file, 'r') as f:
        
        # Parcourt des lignes du fichier.
        for line in f:
            
            # Si la ligne ne commence pas par le caractère #, elle n'est pas utile à la fonction.
            if line[0] != '#':
                
                # Identifiaction des colonnes du fichier.
                fields = line.strip().split('\t')
                
                # Extraction du contenu de la troisième colonne.
                id = fields[2]
                
                # Si la troisième colonne contient le terme "INS", on compte une insertion.
                if 'INS' in id:
                    insertions += 1
                    
                # Dans le cas contraire, on compte une délétion.
                elif 'DEL' in id:
                    deletions += 1
                    
                    
    return (insertions, deletions)  # Voici une cellule pour tester la fonction count_diffs. Elle fait appel à l'initialisation des variables vcf1 et vcf2 deux cellules au dessus.
print("Il y a ", count_indels2(vcf1)[0], " insertions et ", count_indels2(vcf1)[1], " délétions dans vcf1.")
print("Il y a ", count_indels2(vcf2)[0], " insertions et ", count_indels2(vcf2)[1], " délétions dans vcf2.")
  Il y a  14  insertions et  17  délétions dans vcf1.
Il y a  38  insertions et  6  délétions dans vcf2.

Nous avons ensuite réprésenté les résultats sous la forme d'un histogramme obtenu grâce à la fonction suivante et donné en dans le rapport en figure 5.

In [ ]:
# Importation des librairies nécessaires.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
 
# Déclaration et initialisation des données utilisées pour produire l'histogramme.

r = [0,1,2,3]
raw_data = {'orangeBars': [40, 14, 38, 20],'blueBars': [43, 17, 6, 22]} # Ces listes contiennent respectivement le nombre de délétions
                                                                        # et d'insertions présentes à chaque passage de la culture C1.

df = pd.DataFrame(raw_data)
 
# Ici, nous transformons nos données brutes en proportions.

totals = [i+j for i,j in zip(df['orangeBars'], df['blueBars'])]

orangeBars = [i / j for i,j in zip(df['orangeBars'], totals)]
blueBars = [i / j for i,j in zip(df['blueBars'], totals)]
 
# plot

barWidth = 0.85
names = ('P10C1','P20C1','P30C1','P40C1')


# Création des barres qui représentent les proportions d'insertions et de délétions des différents passages.

plt.bar(r, orangeBars, color='#f9bc86', edgecolor='white', width=barWidth, label="Proportion d'insertions")
plt.bar(r, blueBars, bottom=orangeBars, color='#a3acff', edgecolor='white', width=barWidth, label="Proportion de délétions")

# Ajout des pourcentages sur les barres.

for i, (r, o, b) in enumerate(zip(r, orangeBars, blueBars)):
    plt.text(r, o / 2, '{:.0%}'.format(o), ha='center', va='center', color='white', fontsize=10)
    plt.text(r, o + b / 2, '{:.0%}'.format(b), ha='center', va='center', color='white', fontsize=10)


# Etiquettage des axes.

plt.xticks(range(len(names)), names)
plt.xlabel("Passage")

plt.ylabel("Proportion d'insertions et de délétions")
plt.ylim([0, 1])
 
# Création de légende.

plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
 
# Affichage du graphique.

plt.show()

## **Références**

[1] W Shen, S Le, Y Li*, F Hu*. SeqKit: a cross-platform and ultrafast toolkit for FASTA/Q file manipulation. PLOS ONE. doi:10.1371/journal.pone.0163962.

[2] Li, H. (2018). Minimap2: pairwise alignment for nucleotide sequences. Bioinformatics, 34:3094-3100. doi:10.1093/bioinformatics/bty191

[3] Li, H. Samtools. 2023. samtools flagstat – counts the number of alignments for each FLAG type; Disponible sur : http://www.htslib.org/doc/samtools-flagstat.html (Consulté le 03 mars 2023)

[4] Li, H. Samtools. 2023. samtools view – views and converts SAM/BAM/CRAM files. Disponible sur : http://www.htslib.org/doc/samtools-view.html (Consulté le 03 mars 2023)

[5] Li, H. Samtools. 2023. samtools sort – sorts SAM/BAM/CRAM files. Disponible sur : http://www.htslib.org/doc/samtools-sort.html (Consulté le 03 mars 2023)

[6] Li, H. Samtools. 2023. samtools index – indexes SAM/BAM/CRAM files; Disponible sur : http://www.htslib.org/doc/samtools-index.html (Consulté le 03 mars 2023)

[7] Sedlazeck, F.J., Rescheneder, P., Smolka, M. et al. Accurate detection of complex structural variations using single-molecule sequencing. Nat Methods 15, 461–468 (2018). https://doi.org/10.1038/s41592-018-0001-7

[8] Ramírez, Fidel, Devon P. Ryan, Björn Grüning, Vivek Bhardwaj, Fabian Kilpert, Andreas S. Richter, Steffen Heyne, Friederike Dündar, and Thomas Manke. deepTools2: A next Generation Web Server for Deep-Sequencing Data Analysis. Nucleic Acids Research (2016). doi:10.1093/nar/gkw257.

[9] Danecek P, Bonfield JK, Liddle J, Marshall J, Ohan V, Pollard MO, Whitwham A, Keane T, McCarthy SA, Davies RM, Li H. Twelve years of SAMtools and BCFtools. Gigascience. 2021 Feb 16;10(2):giab008. doi: 10.1093/gigascience/giab008. PMID: 33590861; PMCID: PMC7931819.